# Export data to files with Suitcase

In this notebook you will:

* Export saved data to CSV and TIFF.
* See a preview of how exporting data live during a scan works.

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4917, uptime 0:00:34
mini_beamline                    RUNNING   pid 4918, uptime 0:00:34
random_walk                      RUNNING   pid 4919, uptime 0:00:34
random_walk_horiz                RUNNING   pid 4920, uptime 0:00:34
random_walk_vert                 RUNNING   pid 4921, uptime 0:00:34
simple                           RUNNING   pid 4922, uptime 0:00:34
thermo_sim                       RUNNING   pid 4923, uptime 0:00:34
trigger_with_pc                  RUNNING   pid 4924, uptime 0:00:34


In [2]:
%run scripts/beamline_configuration.py

## Scalar data to CSV

In [3]:
RE(scan([ph], motor_ph, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 3     Time: 2020-05-20 15:35:28
Persistent Unique Scan ID: '505ed96a-a883-4a2b-afac-32a36b5e4fff'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+------------+
|   seq_num |       time |   motor_ph |     ph_det |
+-----------+------------+------------+------------+
|         1 | 15:35:28.5 |     -1.000 |      95893 |


|         2 | 15:35:28.6 |     -0.500 |      95901 |
|         3 | 15:35:28.6 |      0.000 |      95965 |
|         4 | 15:35:28.6 |      0.500 |      96205 |
|         5 | 15:35:28.6 |      1.000 |      94113 |
+-----------+------------+------------+------------+
generator scan ['505ed96a'] (scan num: 3)





('505ed96a-a883-4a2b-afac-32a36b5e4fff',)

In [4]:
import suitcase.csv

In [5]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/505ed96a-a883-4a2b-afac-32a36b5e4fff-primary.csv')]}

In [6]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}-{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni-12345-primary.csv')]}

In [7]:
suitcase.csv.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary.csv')]}

## Image data to TIFF

In [8]:
RE(scan([spot], mtr_spotx, -1, 1, 5), sample_comp='Ni', barcode='12345')



Transient Scan ID: 4     Time: 2020-05-20 15:35:28
Persistent Unique Scan ID: '91a7a67b-884a-4426-9a4a-61ecd36e3624'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+-------------+------------+
|   seq_num |       time | motor_spotx |   spot_roi |
+-----------+------------+-------------+------------+
|         1 | 15:35:29.1 |          -1 | 1054787082 |


|         2 | 15:35:29.2 |          -0 | 1053484073 |


|         3 | 15:35:29.3 |           0 | 1054601577 |


|         4 | 15:35:29.4 |           0 | 1056867538 |


|         5 | 15:35:29.5 |           1 | 1060049369 |


+-----------+------------+-------------+------------+
generator scan ['91a7a67b'] (scan num: 4)





('91a7a67b-884a-4426-9a4a-61ecd36e3624',)

### TIFF stack, a 3D block of images in one file

In [9]:
import suitcase.tiff_stack

In [10]:
suitcase.tiff_stack.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345-primary-spot_img.tiff')]}

### TIFF series

In [11]:
import suitcase.tiff_series

Unlike CSV or TIFF stack, this exporter creates separate file per row (Event document). We can include information from a specific Event in the filename. Here, we include the motor position.

In [12]:
suitcase.tiff_series.export(db[-1].documents(fill=True), 'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}/{event[data][motor_spotx]}-')

{'stream_data': [PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-1.0-primary-spot_img-0.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/-0.5-primary-spot_img-1.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.0-primary-spot_img-2.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/0.5-primary-spot_img-3.tiff'),
  PosixPath('/home/travis/build/bluesky/tutorials/docs/source/my_exported_files/Ni/12345/1.0-primary-spot_img-4.tiff')]}

## "Beta" feature: Set up live exporting

*We plan to streamline this in the future, but here is a demonstration that it is possible.*

In [13]:
# Execute this once to configure RE to send data to an exporter.
# At the beamline, this would normally be done in a script automatically at startup.

from event_model import RunRouter
from databroker.core import discover_handlers

def factory(name, start_doc):

    serializer = suitcase.tiff_series.Serializer(
        'my_exported_files/', file_prefix='{start[sample_comp]}/{start[barcode]}-{start[uid]:.8}-')

    def cb(name, doc):
        serializer(name, doc)

    return [cb], []


rr = RunRouter([factory], discover_handlers())
RE.subscribe(rr)

2

In [14]:
RE(count([spot], num=30, delay=1), sample_comp='Lab6', barcode='12345')



Transient Scan ID: 5     Time: 2020-05-20 15:35:29
Persistent Unique Scan ID: '68eff2e1-ca13-4c66-93e7-bc7e1a0f42ec'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

+-----------+------------+------------+
|   seq_num |       time |   spot_roi |
+-----------+------------+------------+
|         1 | 15:35:30.1 | 1079504972 |


|         2 | 15:35:31.0 | 1130968320 |


|         3 | 15:35:32.0 | 1100186873 |


...

|        28 | 15:35:57.0 | 1053626675 |


|        29 | 15:35:58.0 | 1087434869 |


|        30 | 15:35:59.0 | 1131409263 |


+-----------+------------+------------+
generator count ['68eff2e1'] (scan num: 5)





('68eff2e1-ca13-4c66-93e7-bc7e1a0f42ec',)